<a href="https://colab.research.google.com/github/YujiaLIAO-1/housing/blob/main/in_class_assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [140]:
df = pd.read_csv('titanic.csv')

In [141]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [142]:
mean_age = df.Age.mean()
df['Age'] = df['Age'].fillna(mean_age)

df.dropna(subset=['Embarked'], inplace=True)
df.dropna(subset=['Cabin'], inplace=True)

sex_dummies = pd.get_dummies(df['Sex'], dtype=float)
embarked_dummies = pd.get_dummies(df['Embarked'], dtype=float)

df = pd.concat([df, sex_dummies, embarked_dummies], axis=1)


In [143]:
# prompt: 将 'Sex' 列中的女性标记为 1
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'female' else 0)

# prompt: 将 'Embarked' 列中的C标记为 -1, Q记为0， S记为1
df['Embarked'] = df['Embarked'].apply(lambda x: -1 if x == 'C' else (0 if x == 'Q' else 1))

df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female,male,C,Q,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,-1,1.0,0.0,1.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,1,1.0,0.0,0.0,0.0,1.0


In [144]:
X = df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y = df['Survived']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3)


In [145]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
645,1,0,48.000000,1,0,76.7292,-1
298,1,0,29.699118,0,0,30.5000,1
867,1,0,31.000000,0,0,50.4958,1
558,1,1,39.000000,1,1,79.6500,1
516,2,1,34.000000,0,0,10.5000,1


In [146]:
from sklearn.tree import DecisionTreeClassifier

tree_clf1 = DecisionTreeClassifier(criterion='gini', min_samples_split=20)
tree_clf2 = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5)
tree_clf3 = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_split=10)

In [156]:
from sklearn.model_selection import cross_validate
metrics = ['accuracy', 'precision', 'recall', 'f1']
tree_clf1_scores = cross_validate(tree_clf1, X_train, y_train, cv=10, scoring=metrics, return_train_score=True)
tree_clf2_scores = cross_validate(tree_clf2, X_train, y_train, cv=10, scoring=metrics, return_train_score=True)
tree_clf3_scores = cross_validate(tree_clf3, X_train, y_train, cv=10, scoring=metrics, return_train_score=True)

In [160]:
print('Tree 1 average accuracy score:', tree_clf1_scores['test_accuracy'].mean())
print('Tree 2 average accuracy score:', tree_clf2_scores['test_accuracy'].mean())
print('Tree 3 average accuracy score:', tree_clf3_scores['test_accuracy'].mean())

# 找到最优模型的名称
models_scores = {
    'Tree 1': tree_clf1_scores['test_accuracy'].mean(),
    'Tree 2': tree_clf2_scores['test_accuracy'].mean(),
    'Tree 3': tree_clf3_scores['test_accuracy'].mean()
}

best_model_name = max(models_scores, key=models_scores.get)
best_model_accuracy = models_scores[best_model_name]

print(f"The best model based on test accuracy is: {best_model_name} with an accuracy of {best_model_accuracy}")

Tree 1 average accuracy score: 0.7376190476190476
Tree 2 average accuracy score: 0.7723809523809524
Tree 3 average accuracy score: 0.6957142857142856
The best model based on test accuracy is: Tree 2 with an accuracy of 0.7723809523809524


In [161]:
# we train the final model using ALL training data
tree_clf2.fit(X_train, y_train)

# get the prediction results from the testing set
y_pred = tree_clf2.predict(X_test)

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print(f'Accuracy Score : {accuracy_score(y_test, y_pred)}')
print(f'Precision Score : {precision_score(y_test, y_pred)}')
print(f'Recall Score : {recall_score(y_test, y_pred)}')
print(f'F1 Score : {f1_score(y_test, y_pred)}')

Accuracy Score : 0.7868852459016393
Precision Score : 0.9
Recall Score : 0.7297297297297297
F1 Score : 0.8059701492537313
